<a href="https://colab.research.google.com/github/svsvenu/ml-python/blob/main/Create_8821_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt install tesseract-ocr

!pip install datasets pytesseract pdb
!pip install pytesseract datasets shapely lxml
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q datasets seqeval
!pip install accelerate -U




Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 29 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (4,021 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
import sys
sys.path.append('/content')

Mounted at /content/gdrive


In [3]:
import os
import json
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from shapely.geometry import Polygon
import glob
from PIL import Image
from pytesseract import pytesseract
from lxml import etree
import ast
from sklearn.model_selection import train_test_split
pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

In [4]:

from datasets import load_dataset
print ( os.listdir(os.curdir) )
# this dataset uses the new Image feature :)
# dataset = load_dataset("./gdrive/MyDrive/layoutlmv3.py")
dataset = load_dataset("./gdrive/MyDrive/8821layoutlmv3.py")

['.config', 'gdrive', 'sample_data']
The repository for 8821layoutlmv3 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/8821layoutlmv3.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] Y


Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/utils/file_utils.py:1213: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


["{'id': 0, 'file_name': '3db99f5f-f8821_1.jpg', 'tokens': ['venu', 'surampudi', '1007', 'billy', 'allen', 'tx', '75013', '222222222', 'venu', 'surampudi', '1007', 'billy', 'allen', 'tx', '75013', '333333333', 'venu', 'surampudi', '1007', 'billy', 'allen', 'tx', '75013'], 'bboxes': [[151, 544, 225, 562], [239, 538, 405, 570], [152, 579, 222, 602], [235, 578, 300, 611], [151, 618, 224, 642], [235, 621, 263, 642], [276, 619, 365, 642], [1866, 756, 2029, 805], [151, 844, 225, 862], [238, 838, 405, 870], [152, 879, 222, 902], [235, 878, 300, 911], [151, 918, 224, 942], [235, 921, 263, 942], [276, 919, 365, 942], [1866, 1015, 2029, 1038], [151, 1094, 225, 1112], [238, 1088, 405, 1120], [152, 1129, 222, 1152], [235, 1128, 300, 1161], [151, 1168, 224, 1192], [235, 1171, 263, 1192], [276, 1169, 365, 1192]], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 3, 4, 4, 4, 4, 4, 4, 4]}", "{'id': 1, 'file_name': 'ee39c024-f8821_2.jpg', 'tokens': ['venu', 'surampudi', '1007', 'billy', 'allen'

Generating test split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.11/dist-packages/datasets/utils/file_utils.py:1213: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)


["{'id': 3, 'file_name': '09442aba-f8821_4.jpg', 'tokens': ['venu', 'surampudi', '1007', 'billy', 'allen', 'tx', '75013', '222222222', 'venu', 'surampudi', '1007', 'billy', 'allen', 'tx', '75013', '333333333', 'venu', 'surampudi', '1007', 'billy', 'allen', 'tx', '75013'], 'bboxes': [[151, 544, 225, 562], [239, 538, 405, 570], [152, 579, 222, 602], [235, 578, 300, 611], [151, 618, 224, 642], [235, 621, 263, 642], [276, 619, 365, 642], [1866, 756, 2029, 805], [151, 844, 225, 862], [238, 838, 405, 870], [152, 879, 222, 902], [235, 878, 300, 911], [151, 918, 224, 942], [235, 921, 263, 942], [276, 919, 365, 942], [1866, 1015, 2029, 1038], [151, 1094, 225, 1112], [238, 1088, 405, 1120], [152, 1129, 222, 1152], [235, 1128, 300, 1161], [151, 1168, 224, 1192], [235, 1171, 263, 1192], [276, 1169, 365, 1192]], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 3, 4, 4, 4, 4, 4, 4, 4]}", "{'id': 2, 'file_name': '13fdfc6d-f8821_3.jpg', 'tokens': ['venu', 'surampudi', '1007', 'billy', 'allen'

In [5]:
dataset


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'bboxes', 'ner_tags', 'image_path', 'image'],
        num_rows: 2
    })
    test: Dataset({
        features: ['id', 'tokens', 'bboxes', 'ner_tags', 'image_path', 'image'],
        num_rows: 2
    })
})

In [6]:
dataset["train"].features


{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'bboxes': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['tinAndAddress', 'designee1CafNumber', 'designee1NameAndAddres', 'designee2CafNumber', 'designee2NameAndAddres'], id=None), length=-1, id=None),
 'image_path': Value(dtype='string', id=None),
 'image': Image(mode=None, decode=True, id=None)}

In [7]:
example = dataset["train"][0]
example["image"]
qq = example["image"].convert("RGB")
qq.size

(2550, 3300)

In [8]:

qq = example["image"].convert("RGB")
qq.size

(2550, 3300)

In [9]:

from  PIL import Image
qqq = Image.open(r"./gdrive/MyDrive/layoutlmv3/06628fa4-invoice_1.jpg").convert("RGB")
qqq.size


(754, 1000)

In [10]:
words, boxes, ner_tags = example["tokens"], example["bboxes"], example["ner_tags"]
print(len(words)  )
print(len (boxes))
print(len (ner_tags))


23
23
23


In [11]:
from transformers import AutoProcessor
processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/275 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/856 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [12]:
from datasets.features import ClassLabel

features = dataset["train"].features
column_names = dataset["train"].column_names
image_column_name = "image"
text_column_name = "tokens"
boxes_column_name = "bboxes"
label_column_name = "ner_tags"

# In the event the labels are not a `Sequence[ClassLabel]`, we will need to go through the dataset to get the
# unique labels.
def get_label_list(labels):
    unique_labels = set()
    for label in labels:
        unique_labels = unique_labels | set(label)
    label_list = list(unique_labels)
    label_list.sort()
    return label_list

if isinstance(features[label_column_name].feature, ClassLabel):
    label_list = features[label_column_name].feature.names
    # No need to convert the labels since they are already ints.
    id2label = {k: v for k,v in enumerate(label_list)}
    label2id = {v: k for k,v in enumerate(label_list)}
else:
    label_list = get_label_list(dataset["train"][label_column_name])
    id2label = {k: v for k,v in enumerate(label_list)}
    label2id = {v: k for k,v in enumerate(label_list)}
num_labels = len(label_list)

In [13]:
print(label_list)


['tinAndAddress', 'designee1CafNumber', 'designee1NameAndAddres', 'designee2CafNumber', 'designee2NameAndAddres']


In [14]:
print(id2label)


{0: 'tinAndAddress', 1: 'designee1CafNumber', 2: 'designee1NameAndAddres', 3: 'designee2CafNumber', 4: 'designee2NameAndAddres'}


In [15]:

def prepare_examples(examples):
  images = examples[image_column_name]
  words = examples[text_column_name]
  boxes = examples[boxes_column_name]
  word_labels = examples[label_column_name]

  encoding = processor(images, words, boxes=boxes, word_labels=word_labels,
                       truncation=True, padding="max_length")

  return encoding

In [16]:
from datasets import Features, Sequence, ClassLabel, Value, Array2D, Array3D

# we need to define custom features for `set_format` (used later on) to work properly
features = Features({
    'pixel_values': Array3D(dtype="float32", shape=(3, 224, 224)),
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'attention_mask': Sequence(Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'labels': Sequence(feature=Value(dtype='int64')),
})

train_dataset = dataset["train"].map(
    prepare_examples,
    batched=True,
    remove_columns=column_names,
    features=features,
)
eval_dataset = dataset["test"].map(
    prepare_examples,
    batched=True,
    remove_columns=column_names,
    features=features,
)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [17]:
from datasets.features import ClassLabel

features = dataset["train"].features
column_names = dataset["train"].column_names
image_column_name = "image"
text_column_name = "tokens"
boxes_column_name = "bboxes"
label_column_name = "ner_tags"

# In the event the labels are not a `Sequence[ClassLabel]`, we will need to go through the dataset to get the
# unique labels.
def get_label_list(labels):
    unique_labels = set()
    for label in labels:
        unique_labels = unique_labels | set(label)
    label_list = list(unique_labels)
    label_list.sort()
    return label_list

if isinstance(features[label_column_name].feature, ClassLabel):
    label_list = features[label_column_name].feature.names
    # No need to convert the labels since they are already ints.
    id2label = {k: v for k,v in enumerate(label_list)}
    label2id = {v: k for k,v in enumerate(label_list)}
else:
    label_list = get_label_list(dataset["train"][label_column_name])
    id2label = {k: v for k,v in enumerate(label_list)}
    label2id = {v: k for k,v in enumerate(label_list)}
num_labels = len(label_list)



In [18]:
print(label_list)


['tinAndAddress', 'designee1CafNumber', 'designee1NameAndAddres', 'designee2CafNumber', 'designee2NameAndAddres']


In [19]:
print(id2label)


{0: 'tinAndAddress', 1: 'designee1CafNumber', 2: 'designee1NameAndAddres', 3: 'designee2CafNumber', 4: 'designee2NameAndAddres'}


In [20]:
def prepare_examples(examples):
  images = examples[image_column_name]
  words = examples[text_column_name]
  boxes = examples[boxes_column_name]
  word_labels = examples[label_column_name]

  encoding = processor(images, words, boxes=boxes, word_labels=word_labels,
                       truncation=True, padding="max_length")

  return encoding

In [21]:
from datasets import Features, Sequence, ClassLabel, Value, Array2D, Array3D

# we need to define custom features for `set_format` (used later on) to work properly
features = Features({
    'pixel_values': Array3D(dtype="float32", shape=(3, 224, 224)),
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'attention_mask': Sequence(Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'labels': Sequence(feature=Value(dtype='int64')),
})

train_dataset = dataset["train"].map(
    prepare_examples,
    batched=True,
    remove_columns=column_names,
    features=features,
)
eval_dataset = dataset["test"].map(
    prepare_examples,
    batched=True,
    remove_columns=column_names,
    features=features,
)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [22]:
train_dataset


Dataset({
    features: ['pixel_values', 'input_ids', 'attention_mask', 'bbox', 'labels'],
    num_rows: 2
})

In [23]:

example = train_dataset[0]
processor.tokenizer.decode(example["input_ids"])

'<s> venu surampudi 1007 billy allen tx 75013 222222222 venu surampudi 1007 billy allen tx 75013 333333333 venu surampudi 1007 billy allen tx 75013</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

In [24]:
train_dataset.set_format("torch")


In [25]:
import torch

example = train_dataset[0]
for k,v in example.items():
    print(k,v.shape)

pixel_values torch.Size([3, 224, 224])
input_ids torch.Size([512])
attention_mask torch.Size([512])
bbox torch.Size([512, 4])
labels torch.Size([512])


In [26]:
eval_dataset


Dataset({
    features: ['pixel_values', 'input_ids', 'attention_mask', 'bbox', 'labels'],
    num_rows: 2
})

In [27]:
processor.tokenizer.decode(eval_dataset[0]["input_ids"])


'<s> venu surampudi 1007 billy allen tx 75013 222222222 venu surampudi 1007 billy allen tx 75013 333333333 venu surampudi 1007 billy allen tx 75013</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad

In [28]:

for id, label in zip(train_dataset[0]["input_ids"], train_dataset[0]["labels"]):
  print(processor.tokenizer.decode([id]), label.item())

<s> -100
 ven 0
u -100
 sur 0
amp -100
udi -100
 100 0
7 -100
 bill 0
y -100
 all 0
en -100
 tx 0
 75 0
013 -100
 222 1
22 -100
22 -100
22 -100
 ven 2
u -100
 sur 2
amp -100
udi -100
 100 2
7 -100
 bill 2
y -100
 all 2
en -100
 tx 2
 75 2
013 -100
 333 3
3333 -100
33 -100
 ven 4
u -100
 sur 4
amp -100
udi -100
 100 4
7 -100
 bill 4
y -100
 all 4
en -100
 tx 4
 75 4
013 -100
</s> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -100
<pad> -1

In [29]:
!pip install evaluate
import evaluate
metric = evaluate.load('seqeval')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00


In [30]:
import numpy as np

return_entity_level_metrics = False

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

In [31]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test",
                                  max_steps=1000,
                                  per_device_train_batch_size=2,
                                  per_device_eval_batch_size=2,
                                  learning_rate=1e-5,
                                  evaluation_strategy="steps",
                                  eval_steps=100,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="f1")


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1612: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [32]:

from transformers import LayoutLMv3ForTokenClassification

model = LayoutLMv3ForTokenClassification.from_pretrained("microsoft/layoutlmv3-base",
                                                         id2label=id2label,
                                                         label2id=label2id)

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:

from transformers.data.data_collator import default_data_collator

# Initialize our Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics,
)


<ipython-input-33-40bb2881a99e>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [34]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: svsvenu (svsvenu-hash-tag) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1584: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
100,No log,0.001037,1.000000,1.000000,1.000000,1.000000
200,No log,0.000430,1.000000,1.000000,1.000000,1.000000
300,No log,0.000268,1.000000,1.000000,1.000000,1.000000
400,No log,0.000199,1.000000,1.000000,1.000000,1.000000
500,0.048200,0.000161,1.000000,1.000000,1.000000,1.000000
600,0.048200,0.000139,1.000000,1.000000,1.000000,1.000000
700,0.048200,0.000124,1.000000,1.000000,1.000000,1.000000
800,0.048200,0.000116,1.000000,1.000000,1.000000,1.000000
900,0.048200,0.000111,1.000000,1.000000,1.000000,1.000000
1000,0.000300,0.000109,1.000000,1.000000,1.000000,1.000000


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: tinAndAddress seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: designee1CafNumber seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: designee1NameAndAddres seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: designee2CafNumber seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: designee2NameAndAddres seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/lo

TrainOutput(global_step=1000, training_loss=0.024248360246419905, metrics={'train_runtime': 448.9599, 'train_samples_per_second': 4.455, 'train_steps_per_second': 2.227, 'total_flos': 527201236992000.0, 'train_loss': 0.024248360246419905, 'epoch': 1000.0})

In [35]:
trainer.evaluate()


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1584: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: tinAndAddress seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: designee1CafNumber seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: designee1NameAndAddres seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: designee2CafNumber seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: designee2NameAndAddres seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.00010915661550825462,
 'eval_precision': 1.0,
 'eval_recall': 1.0,
 'eval_f1': 1.0,
 'eval_accuracy': 1.0,
 'eval_runtime': 0.2523,
 'eval_samples_per_second': 7.927,
 'eval_steps_per_second': 3.963,
 'epoch': 1000.0}

In [38]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained("/content/test/checkpoint-1000")
torch.save(model, './gdrive/MyDrive/model_pytorch.pt')





In [39]:
import torch
# model = torch.load('./gdrive/MyDrive/model_pytorch.pt')


example = dataset["test"][0]
print(example)


{'id': '0', 'tokens': ['venu', 'surampudi', '1007', 'billy', 'allen', 'tx', '75013', '222222222', 'venu', 'surampudi', '1007', 'billy', 'allen', 'tx', '75013', '333333333', 'venu', 'surampudi', '1007', 'billy', 'allen', 'tx', '75013'], 'bboxes': [[59, 164, 88, 170], [93, 163, 158, 172], [59, 175, 87, 182], [92, 175, 117, 185], [59, 187, 87, 194], [92, 188, 103, 194], [108, 187, 143, 194], [731, 229, 795, 243], [59, 255, 88, 261], [93, 253, 158, 263], [59, 266, 87, 273], [92, 266, 117, 276], [59, 278, 87, 285], [92, 279, 103, 285], [108, 278, 143, 285], [731, 307, 795, 314], [59, 331, 88, 336], [93, 329, 158, 339], [59, 342, 87, 349], [92, 341, 117, 351], [59, 353, 87, 361], [92, 354, 103, 361], [108, 354, 143, 361]], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 3, 4, 4, 4, 4, 4, 4, 4], 'image_path': './gdrive/MyDrive/data/input_images/09442aba-f8821_4.jpg', 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2550x3300 at 0x793FDC662A90>}


In [40]:
image = example["image"]
words = example["tokens"]
boxes = example["bboxes"]
word_labels = example["ner_tags"]

encoding = processor(image, words, boxes=boxes, word_labels=word_labels, return_tensors="pt")
for k,v in encoding.items():
  print(k,v.shape)

print (encoding['labels'] )


input_ids torch.Size([1, 51])
attention_mask torch.Size([1, 51])
bbox torch.Size([1, 51, 4])
labels torch.Size([1, 51])
pixel_values torch.Size([1, 3, 224, 224])
tensor([[-100,    0, -100,    0, -100, -100,    0, -100,    0, -100,    0, -100,
            0,    0, -100,    1, -100, -100, -100,    2, -100,    2, -100, -100,
            2, -100,    2, -100,    2, -100,    2,    2, -100,    3, -100, -100,
            4, -100,    4, -100, -100,    4, -100,    4, -100,    4, -100,    4,
            4, -100, -100]])


In [41]:
import torch

with torch.no_grad():
  outputs = model(**encoding)
logits = outputs.logits
logits.shape
print (outputs)


predictions = logits.argmax(-1).squeeze().tolist()
print(predictions)
labels = encoding.labels.squeeze().tolist()
print(labels)
def unnormalize_box(bbox, width, height):
     return [
         width * (bbox[0] / 1000),
         height * (bbox[1] / 1000),
         width * (bbox[2] / 1000),
         height * (bbox[3] / 1000),
     ]

token_boxes = encoding.bbox.squeeze().tolist()
width, height = image.size

true_predictions = [model.config.id2label[pred] for pred, label in zip(predictions, labels) if label != - 100]
true_labels = [model.config.id2label[label] for prediction, label in zip(predictions, labels) if label != -100]
true_boxes = [unnormalize_box(box, width, height) for box, label in zip(token_boxes, labels) if label != -100]

print(encoding.values)
print(model.config)


TokenClassifierOutput(loss=tensor(0.0001), logits=tensor([[[ 0.2257,  0.6486, -0.0397,  0.2931,  0.2615],
         [ 8.7475, -2.1912, -2.0506, -1.8952, -2.1084],
         [ 8.6328, -2.1752, -1.9337, -1.8350, -2.1814],
         [ 8.7318, -2.1930, -2.0468, -1.8594, -2.1947],
         [ 8.7004, -2.1420, -2.0704, -1.9066, -2.1433],
         [ 8.6305, -2.0426, -2.0767, -1.7978, -2.1776],
         [ 8.7139, -2.2317, -2.0901, -1.9097, -2.1404],
         [ 8.6849, -2.1581, -2.0319, -1.8252, -2.1043],
         [ 8.7455, -2.1820, -2.0580, -1.9398, -2.1482],
         [ 8.6627, -2.1451, -2.0320, -1.9086, -2.0945],
         [ 8.7497, -2.1792, -2.0667, -1.8764, -2.1315],
         [ 8.6943, -2.0925, -2.0477, -1.8972, -2.1075],
         [ 8.6851, -2.2245, -2.0547, -1.9622, -2.1191],
         [ 8.7396, -2.1958, -2.0888, -1.8431, -2.1380],
         [ 8.5793, -2.0911, -2.0938, -1.8950, -2.1377],
         [-1.6156,  8.1529, -1.5011, -1.3438, -1.6235],
         [-1.7847,  7.6867, -1.3593, -0.8237, -1.7085]

In [42]:
logits = outputs.logits
logits.shape
print (outputs)


TokenClassifierOutput(loss=tensor(0.0001), logits=tensor([[[ 0.2257,  0.6486, -0.0397,  0.2931,  0.2615],
         [ 8.7475, -2.1912, -2.0506, -1.8952, -2.1084],
         [ 8.6328, -2.1752, -1.9337, -1.8350, -2.1814],
         [ 8.7318, -2.1930, -2.0468, -1.8594, -2.1947],
         [ 8.7004, -2.1420, -2.0704, -1.9066, -2.1433],
         [ 8.6305, -2.0426, -2.0767, -1.7978, -2.1776],
         [ 8.7139, -2.2317, -2.0901, -1.9097, -2.1404],
         [ 8.6849, -2.1581, -2.0319, -1.8252, -2.1043],
         [ 8.7455, -2.1820, -2.0580, -1.9398, -2.1482],
         [ 8.6627, -2.1451, -2.0320, -1.9086, -2.0945],
         [ 8.7497, -2.1792, -2.0667, -1.8764, -2.1315],
         [ 8.6943, -2.0925, -2.0477, -1.8972, -2.1075],
         [ 8.6851, -2.2245, -2.0547, -1.9622, -2.1191],
         [ 8.7396, -2.1958, -2.0888, -1.8431, -2.1380],
         [ 8.5793, -2.0911, -2.0938, -1.8950, -2.1377],
         [-1.6156,  8.1529, -1.5011, -1.3438, -1.6235],
         [-1.7847,  7.6867, -1.3593, -0.8237, -1.7085]

In [43]:

predictions = logits.argmax(-1).squeeze().tolist()
print(predictions)
labels = encoding.labels.squeeze().tolist()
print(labels)
def unnormalize_box(bbox, width, height):
     return [
         width * (bbox[0] / 1000),
         height * (bbox[1] / 1000),
         width * (bbox[2] / 1000),
         height * (bbox[3] / 1000),
     ]

token_boxes = encoding.bbox.squeeze().tolist()
width, height = image.size

true_predictions = [model.config.id2label[pred] for pred, label in zip(predictions, labels) if label != - 100]
true_labels = [model.config.id2label[label] for prediction, label in zip(predictions, labels) if label != -100]
true_boxes = [unnormalize_box(box, width, height) for box, label in zip(token_boxes, labels) if label != -100]

print(encoding.values)


[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1]
[-100, 0, -100, 0, -100, -100, 0, -100, 0, -100, 0, -100, 0, 0, -100, 1, -100, -100, -100, 2, -100, 2, -100, -100, 2, -100, 2, -100, 2, -100, 2, 2, -100, 3, -100, -100, 4, -100, 4, -100, -100, 4, -100, 4, -100, 4, -100, 4, 4, -100, -100]
<bound method BatchEncoding.values of {'input_ids': tensor([[    0, 21011,   257,  8113,  3914, 24110,   727,   406,  1087,   219,
            70,   225, 48726,  3337, 37129, 25884,  2036,  2036,  2036, 21011,
           257,  8113,  3914, 24110,   727,   406,  1087,   219,    70,   225,
         48726,  3337, 37129, 30242, 46152,  3103, 21011,   257,  8113,  3914,
         24110,   727,   406,  1087,   219,    70,   225, 48726,  3337, 37129,
             2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [44]:
from PIL import ImageDraw, ImageFont

draw = ImageDraw.Draw(image)

font = ImageFont.load_default()

data = pytesseract.image_to_string(image, lang='eng', config='--psm 6')
print( data)

def iob_to_label(label):
    label = label
    if not label:
      return 'other'
    return label

label2color = {'tinandaddress':'blue', 'designee1cafnumber':'green', 'designee1nameandaddres':'orange', 'designee2cafnumber':'yellow', 'designee2nameandaddres':'red'}


for prediction, box in zip(true_predictions, true_boxes):
    predicted_label = iob_to_label(prediction).lower()
    #print(box)
    draw.rectangle(box, outline=label2color[predicted_label])
    draw.text((box[0] + 10, box[1] - 10), text=predicted_label, fill=label2color[predicted_label], font=font)
    cropped_img = image.crop(box)
    cropped_img
    data = pytesseract.image_to_string(cropped_img, lang='eng', config='--psm 6')
    print(predicted_label + "--" + data)

image

# Define the cropping box (left, upper, right, lower)
box = [150.45, 541.2, 224.39999999999998, 561.0]

# Crop the image
cropped_img = image.crop(box)




8821 Tax Information Authorization corine Use bay
Form > Go to www.irs.gov/Form8821 for instructions and the latest information. Received by:
(Rev. January 2021) > Don’t sign this form unless all applicable lines have been completed. Name
> Don’t use Form 8821 to request copies of your tax returns Telephone;
ee ose or to authorize someone to represent you. See instructions. Function
Date
1 Taxpayer information. Taxpayer must sign and date this form on line 6.
Taxpayer name and address Taxpayer identification number(s)
venu surampudi TAA
1007 billy Daytime telephone number | Plan number (if applicable)
allen tx 75013
2 Designee(s). If you wish to name more than two designees, attach a list to this form. Check here if a list of additional
designees is attached» ([]
Name and address CAFNo.  ——___—————isisisi2222222220—
venu surampudi PTIN = ag” a a
1007 billy P TelephoneNo. eee
allen tx 75013 FaxNo.
Check if to be sent copies of notices and communications [_] | Check if new: Address [_] T